--- Day 3: Toboggan Trajectory ---
With the toboggan login problems resolved, you set off toward the airport. While travel by toboggan might be easy, it's certainly not safe: there's very minimal steering and the area is covered in trees. You'll need to see which angles will take you near the fewest trees.

Due to the local geology, trees in this area only grow on exact integer coordinates in a grid. You make a map (your puzzle input) of the open squares (.) and trees (#) you can see. For example:

..##.......  
#...#...#..  
.#....#..#.  
..#.#...#.#  
.#...##..#.  
..#.##.....  
.#.#.#....#  
.#........#  
#.##...#...  
#...##....#  
.#..#...#.#  

These aren't the only trees, though; due to something you read about once involving arboreal genetics and biome stability, the same pattern repeats to the right many times:

..##.........##.........##.........##.........##.........##.......  --->  
#...#...#..#...#...#..#...#...#..#...#...#..#...#...#..#...#...#..  
.#....#..#..#....#..#..#....#..#..#....#..#..#....#..#..#....#..#.  
..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#  
.#...##..#..#...##..#..#...##..#..#...##..#..#...##..#..#...##..#.  
..#.##.......#.##.......#.##.......#.##.......#.##.......#.##.....  --->  
.#.#.#....#.#.#.#....#.#.#.#....#.#.#.#....#.#.#.#....#.#.#.#....#  
.#........#.#........#.#........#.#........#.#........#.#........#  
#.##...#...#.##...#...#.##...#...#.##...#...#.##...#...#.##...#...  
#...##....##...##....##...##....##...##....##...##....##...##....#  
.#..#...#.#.#..#...#.#.#..#...#.#.#..#...#.#.#..#...#.#.#..#...#.#  --->  


You start on the open square (.) in the top-left corner and need to reach the bottom (below the bottom-most row on your map).

The toboggan can only follow a few specific slopes (you opted for a cheaper model that prefers rational numbers); start by counting all the trees you would encounter for the slope right 3, down 1:

From your starting position at the top-left, check the position that is right 3 and down 1. Then, check the position that is right 3 and down 1 from there, and so on until you go past the bottom of the map.

The locations you'd check in the above example are marked here with O where there was an open square and X where there was a tree:

..##.........##.........##.........##.........##.........##.......  --->  
#..O#...#..#...#...#..#...#...#..#...#...#..#...#...#..#...#...#..  
.#....X..#..#....#..#..#....#..#..#....#..#..#....#..#..#....#..#.  
..#.#...#O#..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#  
.#...##..#..X...##..#..#...##..#..#...##..#..#...##..#..#...##..#.  
..#.##.......#.X#.......#.##.......#.##.......#.##.......#.##.....  --->  
.#.#.#....#.#.#.#.O..#.#.#.#....#.#.#.#....#.#.#.#....#.#.#.#....#  
.#........#.#........X.#........#.#........#.#........#.#........#  
#.##...#...#.##...#...#.X#...#...#.##...#...#.##...#...#.##...#...  
#...##....##...##....##...#X....##...##....##...##....##...##....#  
.#..#...#.#.#..#...#.#.#..#...X.#.#..#...#.#.#..#...#.#.#..#...#.#  --->  


In this example, traversing the map using this slope would cause you to encounter 7 trees.

Starting at the top-left corner of your map and following a slope of right 3 and down 1, how many trees would you encounter?

In [3]:
import numpy as np
import re
import math

In [2]:
## puzzle input

inp_loc = './day_3_input.txt'

puzzle_input = []
with open(inp_loc, 'r') as txtfile:
    for line in txtfile:
        puzzle_input.append(line.rstrip())

print('Items:', len(puzzle_input))

print(puzzle_input[0])

print(puzzle_input[-1])

Items: 323
.............#...#....#.....##.
.#...##..##.#.........#.##...#.


In [91]:
## let's turn this into a binary numpy array
## 0: open, 1: tree

trees_array = np.zeros((len(puzzle_input), len(puzzle_input[0])))

for i in range(trees_array.shape[0]):
    tree_locs = [j.start() for j in re.finditer('#', puzzle_input[i])]
    trees_array[i, tree_locs] = 1
    
    
## Now make a function to return how many trees hit

def tree_finder(tree_arr, start_x = 0, slope_x = 1, slope_y = 1):
    """
    Takes a binary numpy array and returns index
    locations where the defined slope encounters
    a `1`. Defaults to identity slope
    
    Additionally, expand array (axis=1) enough times
    to complete slope through bottom row
    """
    tot_columns = tree_arr.shape[0] * slope_x + start_x
    tiles = math.ceil(tot_columns/tree_arr.shape[1])
    tree_arr_expand = np.tile(tree_arr, tiles)
    
    tree_locations = []
    
    for i in range(math.ceil(tree_arr_expand.shape[0]/slope_y)):
        loc0 = i * slope_y
        loc1 = i * slope_x
        if tree_arr_expand[loc0, loc1] == 1:
            tree_locations.append(tuple((loc0, loc1)))
            
    return tree_locations

In [92]:
## get answer for part 1

tree_locs = tree_finder(trees_array, start_x = 0, slope_x = 3, slope_y = 1)

# abswer equals how many trees are in my returned list
print(len(tree_locs))

167


--- Part Two ---
Time to check the rest of the slopes - you need to minimize the probability of a sudden arboreal stop, after all.

Determine the number of trees you would encounter if, for each of the following slopes, you start at the top-left corner and traverse the map all the way to the bottom:

Right 1, down 1.  
Right 3, down 1. (This is the slope you already checked.)  
Right 5, down 1.  
Right 7, down 1.  
Right 1, down 2.  


In the above example, these slopes would find 2, 7, 3, 4, and 2 tree(s) respectively; multiplied together, these produce the answer 336.

What do you get if you multiply together the number of trees encountered on each of the listed slopes?

In [95]:
## get answer for part 2

run1 = tree_finder(trees_array, start_x = 0, slope_x = 1, slope_y = 1)
run2 = tree_finder(trees_array, start_x = 0, slope_x = 3, slope_y = 1)
run3 = tree_finder(trees_array, start_x = 0, slope_x = 5, slope_y = 1)
run4 = tree_finder(trees_array, start_x = 0, slope_x = 7, slope_y = 1)
run5 = tree_finder(trees_array, start_x = 0, slope_x = 1, slope_y = 2)

trees1 = len(run1)
trees2 = len(run2)
trees3 = len(run3)
trees4 = len(run4)
trees5 = len(run5)

print('The number of trees encountered are {}, {}, {}, {}, and {}'.format(trees1, trees2, trees3, trees4, trees5))

ans2 = trees1*trees2*trees3*trees4*trees5
print(ans2)

# trees_arr = [trees1, trees2, trees3, trees4, trees5]
# print(np.product(trees_arr))
# trees_arr = np.array(trees_arr, dtype=np.float64)
# print(np.product(trees_arr))

The number of trees encountered are 53, 167, 54, 67, and 23
736527114


In [93]:
# test case

tst_trees = ['..##.......', '#...#...#..', '.#....#..#.','..#.#...#.#','.#...##..#.','..#.##.....',
            '.#.#.#....#', '.#........#', '#.##...#...', '#...##....#', '.#..#...#.#']

tst_array = np.zeros((len(tst_trees), len(tst_trees[0])))

for i in range(tst_array.shape[0]):
    tree_locs = [j.start() for j in re.finditer('#', tst_trees[i])]
    tst_array[i, tree_locs] = 1
    
print(tst_array)

tst1 = tree_finder(tst_array, start_x = 0, slope_x = 1, slope_y = 1)
tst2 = tree_finder(tst_array, start_x = 0, slope_x = 3, slope_y = 1)
tst3 = tree_finder(tst_array, start_x = 0, slope_x = 5, slope_y = 1)
tst4 = tree_finder(tst_array, start_x = 0, slope_x = 7, slope_y = 1)
tst5 = tree_finder(tst_array, start_x = 0, slope_x = 1, slope_y = 2) 

print('The number of trees encountered are {}, {}, {}, {}, and {}'.format(len(tst1), len(tst2), len(tst3), len(tst4), len(tst5)))

tst_trees = np.array([len(tst1), len(tst2), len(tst3), len(tst4), len(tst5)], dtype=np.float64)
tst_ans = np.prod(tst_trees)
print(tst_ans)

[[0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0.]
 [0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1.]
 [0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0.]
 [0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1.]]
The number of trees encountered are 2, 7, 3, 4, and 2
336.0
